# SSH on CHI@Edge containers

In [1]:
import chi
# Before we go any further, we need to select which Chameleon site we will be using.
chi.use_site("CHI@Edge")

Now using CHI@Edge:
URL: https://chi.edge.chameleoncloud.org
Location: University of Chicago, Chicago, Illinois, USA
Support contact: help@chameleoncloud.org


In [2]:
chi.set("project_name", "Chameleon")

In [3]:
from chi import container
from chi import lease

## Creating a 1-day lease

We are creating a short 1 day lease for a raspberry pi 4 device 

In [4]:
# machine name refers to the "type" of device
machine_name = "raspberrypi4-64"

# get dates for lease start and end
start, end = lease.lease_duration(days=1)

# make a unique name for the lease
lease_name = f"ssh-{machine_name}-{start}"

reservations = []
lease.add_device_reservation(reservations, count=2, machine_name=machine_name)
container_lease = lease.create_lease(lease_name, reservations)
lease_id = container_lease["id"]

print(f"created lease with name {lease_name} and uuid {lease_id}, waiting for it to start. This can take up to 60s.")
lease.wait_for_active(lease_id)
print("Done!")

created lease with name ssh-raspberrypi4-64-2024-05-01 15:05 and uuid 205c57ee-8451-428d-aaa1-0f30bc917ffe, waiting for it to start. This can take up to 60s.
Done!


## Starting the container

The image we are using is a base [ubuntu 22.04 image](https://github.com/ChameleonCloud/edge_ssh_image) where we install a variety of utilities and apply some modifications to the sshd config before starting the sshd daemon. 

In [5]:
print("Requesting container ... This may take a while as the large container image is being downloaded")

# Set a name for the container. Because CHI@Edge uses Kubernetes, ensure that underscores aren't in the name
container_name = f"tutorial-{machine_name}-ssh".replace("_","-")

try:
    my_container = container.create_container(
        container_name, 
        image="ghcr.io/chameleoncloud/edge_ssh_image:latest",
        workdir="/home",
        exposed_ports=[22],
        reservation_id=lease.get_device_reservation(lease_id),
        platform_version=2,
    )
except RuntimeError as ex:
    print(ex)
    print(f"please stop and/or delete {container_name} and try again")
else:
    print(f"Successfully created container: {container_name}!")

Requesting container ... This may take a while as the large container image is being downloaded
Successfully created container: tutorial-raspberrypi4-64-ssh!


## Adding your public key to the authorized keys within the container

**Important** Please first place your RSA public key in one line in the 'edge_user_public_key' text file. The following cells will then upload the file to the running container, and copy over the key to the authorized keys file after setting the appropriate permissions.

In [6]:
chi.container.upload(my_container.uuid, "./edge_user_public_key", "/root/.ssh/")

(<Response [200]>, None)

In [7]:
cmd = "/bin/bash -c \"chmod 600 /root/.ssh/authorized_keys\""
chi.container.execute(my_container.uuid, cmd)

{'output': '', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}

In [8]:
cmd = "/bin/bash -c \"cat /root/.ssh/edge_user_public_key >> /root/.ssh/authorized_keys\""
chi.container.execute(my_container.uuid, cmd)

{'output': '', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}

## SSH'ing into the container

if ssh'ing into the device hangs forever when connecting to device part, please release the floating IP via the chi@edge openstack dashboard and rerun the cell below

In [10]:
ip_address = chi.container.associate_floating_ip(my_container.uuid)

print("use the following command to ssh into the container: ssh root@" + ip_address)

use the following command to ssh into the container: ssh root@129.114.34.187
